# 🔤 A medium to communicate with the LLMs.

### 📜 A simple back story
Till now we have been working with the **character level** tokenizer which was very *naive* and actually worked pretty well! But in the real world the story of tokenzer is a little different.

So, as a recap:
1. Gather all characters (letters, numbers, special characters).
2. Assign each a unique index.
3. Convert characters to indices.
4. Form a lookup table (embeddings).
5. The table has `n` rows and `m` columns (where `m` may equal `n` for simple lookup).
6. Train the model.
8. **Pluck out the corresponding row** from the lookup table for a token.
9. Done!

### ⌚ Now
Now, we are going to create the **chunk-level** tokenizer. And the most common algorithm is **Byte-pair encoding algorithm** *(which is very simple, indeed 😉)*.

# 🤪 Some tokenizer weirdo
There is a lot going on with tokenizers. As Andrej shares in his notebook, I will share some of my exploration as well. But first, let's just copy-paste his list of weirdos.

Tokenization is at the heart of much weirdness of LLMs. Do not brush it off.

- **Can't LLM spell** words? 
- **Can't LLM do super simple string processing** tasks like reversing a string?.
    - Ex: Reverse the string "lollipop"
- Why is LLM **worse at non-English** languages (e.g. Japanese)?
- Why is LLM **bad at simple arithmetic**?
- GPT-2 have **more** than necessary **trouble** coding in Python?
- Why did my LLM **abruptly halt** when it sees the string `<|endoftext|>`?
- What is this weird warning I get about a "trailing whitespace"? **Tokenization**.
- Why the LLM break if I ask it about `SolidGoldMagikarp`?
- Why should I prefer to use YAML over JSON with LLMs?
- Why is LLM not actually end-to-end language modeling?
- What is the real root of suffering? **Tokenization**.

Well, the `SolidGoldMagikrap` was amazing to look at. Please watch the [video here](https://youtu.be/WO2X3oZEJOA).

📝 But the summary is:
- `SolidGoldMagikrap` is a whole token.
- That doesn't mean anything on its own.
- That **is actually a username** of the redditer.
- It was so repetative, that the LLM got more count than necessary that became **a whole** token to the model.
- When telling the earlier versions of the LLM to say `SolidGoldMagikrap`, the LLM used to give very different answer. Very very weird answer.

All because of the **tokenization**.

### 🪀 Some tokenizers to play around
1. https://tiktokenizer.vercel.app/
2. https://platform.openai.com/tokenizer

### 👍🏻 Advantages of "longer tokens"
- Saves space in the context window.
- Less time for the model to learn basic connection/meaning of tokens (as now whole token represents something instead of its atomic terms)
- Can fit more information in the same context window than its conterpart.

### 👎🏻 Disadvantages
- Need to keep track of all combinations (not all, but most of).
- Increases vocab size.

> *Still making the vocab size infinitely larger is not a good choice, as it will impact the softmax function and the prediction of the next token may not be optimal - unless you have a very very large training dataset*.

# 🌌 The Unicode-verse
Now, let's talk about the "translation" from from the ABC into 1, 2, 3 (the numerical representation).

- What we did previously was to tokenize the vocabulary and give them individual index **manually**.
- Python natively has *vocabulary support*.
- So, if you endup using the `ord()` function, it will return you the *character level* index.

For an example `ord("A")` is `65` and `ord("👍")` is `128077`.
___

### 🚫 But we can't use that.
The unicode mapping keep changing over time, and we don't know if today's `65` become `223` tomorrow! And this will f** up the model's understanding.

### 🔢 Something about the UTF-8, 16, and 32
📃 In the spare time, do read: https://www.reedbeta.com/blog/programmers-intro-to-unicode/#the-unicode-codespace

The hitline is:
> UTF-8, 16 and 32 are just the **representations** and not the storages. They all store all characters of unicode. They just tell *how* the bytes are represented.

**Example:**
- Say for an example `a` has the number `97`. So, when we encode that with:
```python
>>>list("a".encode("utf-16"))
[97]

>>> list("a".encode("utf-16"))
[255, 254, 97, 0, 98, 0]

>>> list("a".encode("utf-32"))
[255, 254, 0, 0, 97, 0, 0, 0]
```

What happens is: All characters are already in the unicode database, but when you want to **represent** them, you select some sort of encoding, and that represents the stuff as needed.

> 💬
>
> **A single byte can only store upto `256` individual characters**. The game is representing the stuff with the combinations of these bytes.
>
> — Aayush

In the case of `a`, it was in the range of 256 so utf-8 could represent that with the single byte (8bits == 1 byte, and 32bits == 4bytes okay?).

But with the utf-16 and utf-32 respectively, they want to add the additional zeros `0` to **support their structure.**

Since `97` can be represented with a single byte, no need other bytes. And thus, utf-16 gave `[255, 254, 97, 0]` where `254, 97` are 2 bytes (16bits) and with utf-32 `[255, 254, 0, 0, 97, 0, 0, 0]` where `254, 0, 0, 97` are 4 bytes (32bits).

The `254` and `255` represented the starting of the string characters. That is the structure, and also the remaining `0, 0, 0` represent the ending. Which are required for its internal representation.

___

Once we go beyond the `256` limit, we will be able to see the difference.

___

### Whtever, 
The point here is, **when we will use encoding, depending on the combinations *(even utf-8 will make the combinations when the character is out of its reach)* we will have a large byte stream to process**.

> 💬
> 
> *But we don't want to use the **raw** bytes of the utf-8 encoding, we want to be able to support larger vocabulary size that we can tune as a hyperparameter (like making larger or smaller vocab) **but also** we want to stick with the utf-8 encoding.*
>
> — Andrej


# 1️⃣0️⃣0️⃣1️⃣ BPE: Byte Pair Encoding

It is:
> *To iteratively find the pair of tokens that occur most frequently*.

👉🏻 Here is a quick explanation from the [Wikipedia](https://en.wikipedia.org/wiki/Byte_pair_encoding):

![BPE-Intro](./images/bpe-intro.png)

**Quick takeaways:**
- To build the vocab, we will need to run multiplepasses over the whole datasets
- There, we will need to get the "most common pair".
- Where **pair** means just two tokens together. Just the single-single.
- Doing that iteratively will create a whole vocab.

🔗 Andrej explains that easily in [this clip](https://youtube.com/clip/UgkxSxfurF6u8ju67A6ZJ8mxVd6V_A_ljUBL?si=hP9Ku8SXyxIExdsm).

# 🔥 Starting the BPE

In [1]:
# Let's see the example text in action
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."

tokens = text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience
print(text)
print("👉🏻 LENGTH:", len(text))
print('---')
print(tokens)
print("👉🏻 LENGTH:", len(tokens))

Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.
👉🏻 LENGTH: 533
---
[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 128, 140,

**The length** of the **encoded** string is now increased. And that is because of how the UTF-8 encodes the stuff. Right. Now, let's start working on the algorithm.

In [2]:
from collections import defaultdict

### 1️⃣ Step: Count

In [3]:
def get_max_occuring_pair_and_stats(tokens):
    counter = defaultdict(int)
    for pair in zip(tokens, tokens[1:]):
        counter[pair] += 1
    
    max_pair = max(counter, key=counter.get) # will use te get function for all keys and give the max count pair
    return max_pair, counter[max_pair], counter

In [4]:
pair, count, stats = get_max_occuring_pair_and_stats(tokens)

In [5]:
pair

(101, 32)

In [6]:
count

20

This is correct! As the currently the tokens has `101 32` as the most occuring pair. Which correspond to:

In [7]:
chr(101), chr(32)

('e', ' ')

### 2️⃣ Step: Replace

In [8]:
def replace(tokens, pair, new_id):
    '''
    tokens: The list of tokens to update
    pair: the pair to replace
    new_id: the id to replace the pair with

    See, there will be **multiple locations** of the pair that can exist 
    in the span. So, we will need to iterate over the list of these indexes
    and then where found, just hit replace.
    '''

    # the new place for the updated tokens
    new_tokens = []
    i = 0
    while i < len(tokens):
        # the current position has the matching pair -> replace
        if (i < len(tokens) - 1) and (tokens[i] == pair[0]) and (tokens[i+1] == pair[1]):
            new_tokens.append(new_id)
            i += 2 # incrementing i by 2 because we have replace 2 tokens!

        # means, we have non matching token with the pair
        else:
            new_tokens.append(tokens[i])
            i += 1 # increament by 1 only as no replacement has been made
    return new_tokens

In [9]:
# A toy example
replace(tokens = [1, 33,455, 4, 22, 5, 4, 22], 
        pair = (4, 22), 
        new_id = 422)

[1, 33, 455, 422, 5, 422]

### 3️⃣ Step: Iteratively work

In [10]:
def apply_bpe(tokens):
    tokens_to_compress = tokens.copy()
    new_id = 255
    new_tokens_mapping = {}
    while True:
        max_pair, count, _ = get_max_occuring_pair_and_stats(tokens=tokens_to_compress)
        if count != 1:
            new_id += 1
            tokens_to_compress = replace(tokens=tokens_to_compress, pair=max_pair, new_id=new_id)
            new_tokens_mapping[max_pair] = new_id
        else:
            return tokens_to_compress, new_tokens_mapping

In [11]:
new_tokens, mapping = apply_bpe(tokens)

In [12]:
len(new_tokens)

300

In [13]:
mapping

{(101, 32): 256,
 (240, 159): 257,
 (226, 128): 258,
 (105, 110): 259,
 (115, 32): 260,
 (97, 110): 261,
 (116, 104): 262,
 (257, 133): 263,
 (257, 135): 264,
 (97, 114): 265,
 (239, 189): 266,
 (258, 140): 267,
 (267, 264): 268,
 (101, 114): 269,
 (111, 114): 270,
 (116, 32): 271,
 (259, 103): 272,
 (115, 116): 273,
 (261, 100): 274,
 (32, 262): 275,
 (44, 32): 276,
 (97, 109): 277,
 (275, 256): 278,
 (111, 117): 279,
 (85, 110): 280,
 (280, 105): 281,
 (281, 99): 282,
 (282, 111): 283,
 (283, 100): 284,
 (115, 276): 285,
 (273, 114): 286,
 (101, 265): 287,
 (274, 32): 288,
 (259, 116): 289,
 (111, 102): 290,
 (46, 32): 291,
 (108, 108): 292,
 (272, 32): 293,
 (261, 32): 294,
 (101, 110): 295,
 (33, 32): 296,
 (118, 269): 297,
 (121, 32): 298,
 (277, 256): 299,
 (105, 107): 300,
 (101, 260): 301,
 (119, 256): 302,
 (289, 111): 303,
 (303, 278): 304,
 (116, 260): 305,
 (290, 32): 306,
 (112, 114): 307,
 (307, 111): 308,
 (308, 103): 309,
 (309, 114): 310,
 (310, 277): 311,
 (311, 109):

### 4️⃣ Step: Define the vocab size

Now, the current setting will create all possible pairs, but **at the cost of** very large vocab size.

In [14]:
vocab_size = 276
num_merges = vocab_size - 256 # total 20 merges will happen

def apply_bpe(tokens):
    tokens_to_compress = tokens.copy()
    new_id = 255
    new_tokens_mapping = {}
    for i in range(num_merges):
        max_pair, count, _ = get_max_occuring_pair_and_stats(tokens=tokens_to_compress)
        new_id += 1
        tokens_to_compress = replace(tokens=tokens_to_compress, pair=max_pair, new_id=new_id)
        new_tokens_mapping[max_pair] = new_id    
    return tokens_to_compress, new_tokens_mapping

In [15]:
new_tokens, mapping = apply_bpe(tokens)

In [16]:
len(new_tokens)

451

In [17]:
len(mapping.keys())

20

Cool! 🔥

Now, we have limited the number of new tokens that can be added. Here, only `20` new additions were made.

###  5️⃣ Step: On whole article!

In [18]:
# making the training text longer to have more representative token statistics
# text from https://www.reedbeta.com/blog/programmers-intro-to-unicode/
text = """A Programmer’s Introduction to Unicode March 3, 2017 · Coding · 22 Comments  Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺\u200c🇳\u200c🇮\u200c🇨\u200c🇴\u200c🇩\u200c🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.  A few months ago, I got interested in Unicode and decided to spend some time learning more about it in detail. In this article, I’ll give an introduction to it from a programmer’s point of view.  I’m going to focus on the character set and what’s involved in working with strings and files of Unicode text. However, in this article I’m not going to talk about fonts, text layout/shaping/rendering, or localization in detail—those are separate issues, beyond my scope (and knowledge) here.  Diversity and Inherent Complexity The Unicode Codespace Codespace Allocation Scripts Usage Frequency Encodings UTF-8 UTF-16 Combining Marks Canonical Equivalence Normalization Forms Grapheme Clusters And More… Diversity and Inherent Complexity As soon as you start to study Unicode, it becomes clear that it represents a large jump in complexity over character sets like ASCII that you may be more familiar with. It’s not just that Unicode contains a much larger number of characters, although that’s part of it. Unicode also has a great deal of internal structure, features, and special cases, making it much more than what one might expect a mere “character set” to be. We’ll see some of that later in this article.  When confronting all this complexity, especially as an engineer, it’s hard not to find oneself asking, “Why do we need all this? Is this really necessary? Couldn’t it be simplified?”  However, Unicode aims to faithfully represent the entire world’s writing systems. The Unicode Consortium’s stated goal is “enabling people around the world to use computers in any language”. And as you might imagine, the diversity of written languages is immense! To date, Unicode supports 135 different scripts, covering some 1100 languages, and there’s still a long tail of over 100 unsupported scripts, both modern and historical, which people are still working to add.  Given this enormous diversity, it’s inevitable that representing it is a complicated project. Unicode embraces that diversity, and accepts the complexity inherent in its mission to include all human writing systems. It doesn’t make a lot of trade-offs in the name of simplification, and it makes exceptions to its own rules where necessary to further its mission.  Moreover, Unicode is committed not just to supporting texts in any single language, but also to letting multiple languages coexist within one text—which introduces even more complexity.  Most programming languages have libraries available to handle the gory low-level details of text manipulation, but as a programmer, you’ll still need to know about certain Unicode features in order to know when and how to apply them. It may take some time to wrap your head around it all, but don’t be discouraged—think about the billions of people for whom your software will be more accessible through supporting text in their language. Embrace the complexity!  The Unicode Codespace Let’s start with some general orientation. The basic elements of Unicode—its “characters”, although that term isn’t quite right—are called code points. Code points are identified by number, customarily written in hexadecimal with the prefix “U+”, such as U+0041 “A” latin capital letter a or U+03B8 “θ” greek small letter theta. Each code point also has a short name, and quite a few other properties, specified in the Unicode Character Database.  The set of all possible code points is called the codespace. The Unicode codespace consists of 1,114,112 code points. However, only 128,237 of them—about 12% of the codespace—are actually assigned, to date. There’s plenty of room for growth! Unicode also reserves an additional 137,468 code points as “private use” areas, which have no standardized meaning and are available for individual applications to define for their own purposes.  Codespace Allocation To get a feel for how the codespace is laid out, it’s helpful to visualize it. Below is a map of the entire codespace, with one pixel per code point. It’s arranged in tiles for visual coherence; each small square is 16×16 = 256 code points, and each large square is a “plane” of 65,536 code points. There are 17 planes altogether.  Map of the Unicode codespace (click to zoom)  White represents unassigned space. Blue is assigned code points, green is private-use areas, and the small red area is surrogates (more about those later). As you can see, the assigned code points are distributed somewhat sparsely, but concentrated in the first three planes.  Plane 0 is also known as the “Basic Multilingual Plane”, or BMP. The BMP contains essentially all the characters needed for modern text in any script, including Latin, Cyrillic, Greek, Han (Chinese), Japanese, Korean, Arabic, Hebrew, Devanagari (Indian), and many more.  (In the past, the codespace was just the BMP and no more—Unicode was originally conceived as a straightforward 16-bit encoding, with only 65,536 code points. It was expanded to its current size in 1996. However, the vast majority of code points in modern text belong to the BMP.)  Plane 1 contains historical scripts, such as Sumerian cuneiform and Egyptian hieroglyphs, as well as emoji and various other symbols. Plane 2 contains a large block of less-common and historical Han characters. The remaining planes are empty, except for a small number of rarely-used formatting characters in Plane 14; planes 15–16 are reserved entirely for private use.  Scripts Let’s zoom in on the first three planes, since that’s where the action is:  Map of scripts in Unicode planes 0–2 (click to zoom)  This map color-codes the 135 different scripts in Unicode. You can see how Han () and Korean () take up most of the range of the BMP (the left large square). By contrast, all of the European, Middle Eastern, and South Asian scripts fit into the first row of the BMP in this diagram.  Many areas of the codespace are adapted or copied from earlier encodings. For example, the first 128 code points of Unicode are just a copy of ASCII. This has clear benefits for compatibility—it’s easy to losslessly convert texts from smaller encodings into Unicode (and the other direction too, as long as no characters outside the smaller encoding are used).  Usage Frequency One more interesting way to visualize the codespace is to look at the distribution of usage—in other words, how often each code point is actually used in real-world texts. Below is a heat map of planes 0–2 based on a large sample of text from Wikipedia and Twitter (all languages). Frequency increases from black (never seen) through red and yellow to white.  Heat map of code point usage frequency in Unicode planes 0–2 (click to zoom)  You can see that the vast majority of this text sample lies in the BMP, with only scattered usage of code points from planes 1–2. The biggest exception is emoji, which show up here as the several bright squares in the bottom row of plane 1.  Encodings We’ve seen that Unicode code points are abstractly identified by their index in the codespace, ranging from U+0000 to U+10FFFF. But how do code points get represented as bytes, in memory or in a file?  The most convenient, computer-friendliest (and programmer-friendliest) thing to do would be to just store the code point index as a 32-bit integer. This works, but it consumes 4 bytes per code point, which is sort of a lot. Using 32-bit ints for Unicode will cost you a bunch of extra storage, memory, and performance in bandwidth-bound scenarios, if you work with a lot of text.  Consequently, there are several more-compact encodings for Unicode. The 32-bit integer encoding is officially called UTF-32 (UTF = “Unicode Transformation Format”), but it’s rarely used for storage. At most, it comes up sometimes as a temporary internal representation, for examining or operating on the code points in a string.  Much more commonly, you’ll see Unicode text encoded as either UTF-8 or UTF-16. These are both variable-length encodings, made up of 8-bit or 16-bit units, respectively. In these schemes, code points with smaller index values take up fewer bytes, which saves a lot of memory for typical texts. The trade-off is that processing UTF-8/16 texts is more programmatically involved, and likely slower.  UTF-8 In UTF-8, each code point is stored using 1 to 4 bytes, based on its index value.  UTF-8 uses a system of binary prefixes, in which the high bits of each byte mark whether it’s a single byte, the beginning of a multi-byte sequence, or a continuation byte; the remaining bits, concatenated, give the code point index. This table shows how it works:  UTF-8 (binary)\tCode point (binary)\tRange 0xxxxxxx\txxxxxxx\tU+0000–U+007F 110xxxxx 10yyyyyy\txxxxxyyyyyy\tU+0080–U+07FF 1110xxxx 10yyyyyy 10zzzzzz\txxxxyyyyyyzzzzzz\tU+0800–U+FFFF 11110xxx 10yyyyyy 10zzzzzz 10wwwwww\txxxyyyyyyzzzzzzwwwwww\tU+10000–U+10FFFF A handy property of UTF-8 is that code points below 128 (ASCII characters) are encoded as single bytes, and all non-ASCII code points are encoded using sequences of bytes 128–255. This has a couple of nice consequences. First, any strings or files out there that are already in ASCII can also be interpreted as UTF-8 without any conversion. Second, lots of widely-used string programming idioms—such as null termination, or delimiters (newlines, tabs, commas, slashes, etc.)—will just work on UTF-8 strings. ASCII bytes never occur inside the encoding of non-ASCII code points, so searching byte-wise for a null terminator or a delimiter will do the right thing.  Thanks to this convenience, it’s relatively simple to extend legacy ASCII programs and APIs to handle UTF-8 strings. UTF-8 is very widely used in the Unix/Linux and Web worlds, and many programmers argue UTF-8 should be the default encoding everywhere.  However, UTF-8 isn’t a drop-in replacement for ASCII strings in all respects. For instance, code that iterates over the “characters” in a string will need to decode UTF-8 and iterate over code points (or maybe grapheme clusters—more about those later), not bytes. When you measure the “length” of a string, you’ll need to think about whether you want the length in bytes, the length in code points, the width of the text when rendered, or something else.  UTF-16 The other encoding that you’re likely to encounter is UTF-16. It uses 16-bit words, with each code point stored as either 1 or 2 words.  Like UTF-8, we can express the UTF-16 encoding rules in the form of binary prefixes:  UTF-16 (binary)\tCode point (binary)\tRange xxxxxxxxxxxxxxxx\txxxxxxxxxxxxxxxx\tU+0000–U+FFFF 110110xxxxxxxxxx 110111yyyyyyyyyy\txxxxxxxxxxyyyyyyyyyy + 0x10000\tU+10000–U+10FFFF A more common way that people talk about UTF-16 encoding, though, is in terms of code points called “surrogates”. All the code points in the range U+D800–U+DFFF—or in other words, the code points that match the binary prefixes 110110 and 110111 in the table above—are reserved specifically for UTF-16 encoding, and don’t represent any valid characters on their own. They’re only meant to occur in the 2-word encoding pattern above, which is called a “surrogate pair”. Surrogate code points are illegal in any other context! They’re not allowed in UTF-8 or UTF-32 at all.  Historically, UTF-16 is a descendant of the original, pre-1996 versions of Unicode, in which there were only 65,536 code points. The original intention was that there would be no different “encodings”; Unicode was supposed to be a straightforward 16-bit character set. Later, the codespace was expanded to make room for a long tail of less-common (but still important) Han characters, which the Unicode designers didn’t originally plan for. Surrogates were then introduced, as—to put it bluntly—a kludge, allowing 16-bit encodings to access the new code points.  Today, Javascript uses UTF-16 as its standard string representation: if you ask for the length of a string, or iterate over it, etc., the result will be in UTF-16 words, with any code points outside the BMP expressed as surrogate pairs. UTF-16 is also used by the Microsoft Win32 APIs; though Win32 supports either 8-bit or 16-bit strings, the 8-bit version unaccountably still doesn’t support UTF-8—only legacy code-page encodings, like ANSI. This leaves UTF-16 as the only way to get proper Unicode support in Windows. (Update: in Win10 version 1903, they finally added UTF-8 support to the 8-bit APIs! 😊)  By the way, UTF-16’s words can be stored either little-endian or big-endian. Unicode has no opinion on that issue, though it does encourage the convention of putting U+FEFF zero width no-break space at the top of a UTF-16 file as a byte-order mark, to disambiguate the endianness. (If the file doesn’t match the system’s endianness, the BOM will be decoded as U+FFFE, which isn’t a valid code point.)  Combining Marks In the story so far, we’ve been focusing on code points. But in Unicode, a “character” can be more complicated than just an individual code point!  Unicode includes a system for dynamically composing characters, by combining multiple code points together. This is used in various ways to gain flexibility without causing a huge combinatorial explosion in the number of code points.  In European languages, for example, this shows up in the application of diacritics to letters. Unicode supports a wide range of diacritics, including acute and grave accents, umlauts, cedillas, and many more. All these diacritics can be applied to any letter of any alphabet—and in fact, multiple diacritics can be used on a single letter.  If Unicode tried to assign a distinct code point to every possible combination of letter and diacritics, things would rapidly get out of hand. Instead, the dynamic composition system enables you to construct the character you want, by starting with a base code point (the letter) and appending additional code points, called “combining marks”, to specify the diacritics. When a text renderer sees a sequence like this in a string, it automatically stacks the diacritics over or under the base letter to create a composed character.  For example, the accented character “Á” can be expressed as a string of two code points: U+0041 “A” latin capital letter a plus U+0301 “◌́” combining acute accent. This string automatically gets rendered as a single character: “Á”.  Now, Unicode does also include many “precomposed” code points, each representing a letter with some combination of diacritics already applied, such as U+00C1 “Á” latin capital letter a with acute or U+1EC7 “ệ” latin small letter e with circumflex and dot below. I suspect these are mostly inherited from older encodings that were assimilated into Unicode, and kept around for compatibility. In practice, there are precomposed code points for most of the common letter-with-diacritic combinations in European-script languages, so they don’t use dynamic composition that much in typical text.  Still, the system of combining marks does allow for an arbitrary number of diacritics to be stacked on any base character. The reductio-ad-absurdum of this is Zalgo text, which works by ͖͟ͅr͞aṋ̫̠̖͈̗d͖̻̹óm̪͙͕̗̝ļ͇̰͓̳̫ý͓̥̟͍ ̕s̫t̫̱͕̗̰̼̘͜a̼̩͖͇̠͈̣͝c̙͍k̖̱̹͍͘i̢n̨̺̝͇͇̟͙ģ̫̮͎̻̟ͅ ̕n̼̺͈͞u̮͙m̺̭̟̗͞e̞͓̰̤͓̫r̵o̖ṷs҉̪͍̭̬̝̤ ̮͉̝̞̗̟͠d̴̟̜̱͕͚i͇̫̼̯̭̜͡ḁ͙̻̼c̲̲̹r̨̠̹̣̰̦i̱t̤̻̤͍͙̘̕i̵̜̭̤̱͎c̵s ͘o̱̲͈̙͖͇̲͢n͘ ̜͈e̬̲̠̩ac͕̺̠͉h̷̪ ̺̣͖̱ḻ̫̬̝̹ḙ̙̺͙̭͓̲t̞̞͇̲͉͍t̷͔̪͉̲̻̠͙e̦̻͈͉͇r͇̭̭̬͖,̖́ ̜͙͓̣̭s̘̘͈o̱̰̤̲ͅ ̛̬̜̙t̼̦͕̱̹͕̥h̳̲͈͝ͅa̦t̻̲ ̻̟̭̦̖t̛̰̩h̠͕̳̝̫͕e͈̤̘͖̞͘y҉̝͙ ̷͉͔̰̠o̞̰v͈͈̳̘͜er̶f̰͈͔ḻ͕̘̫̺̲o̲̭͙͠ͅw̱̳̺ ͜t̸h͇̭͕̳͍e̖̯̟̠ ͍̞̜͔̩̪͜ļ͎̪̲͚i̝̲̹̙̩̹n̨̦̩̖ḙ̼̲̼͢ͅ ̬͝s̼͚̘̞͝p͙̘̻a̙c҉͉̜̤͈̯̖i̥͡n̦̠̱͟g̸̗̻̦̭̮̟ͅ ̳̪̠͖̳̯̕a̫͜n͝d͡ ̣̦̙ͅc̪̗r̴͙̮̦̹̳e͇͚̞͔̹̫͟a̙̺̙ț͔͎̘̹ͅe̥̩͍ a͖̪̜̮͙̹n̢͉̝ ͇͉͓̦̼́a̳͖̪̤̱p̖͔͔̟͇͎͠p̱͍̺ę̲͎͈̰̲̤̫a̯͜r̨̮̫̣̘a̩̯͖n̹̦̰͎̣̞̞c̨̦̱͔͎͍͖e̬͓͘ ̤̰̩͙̤̬͙o̵̼̻̬̻͇̮̪f̴ ̡̙̭͓͖̪̤“̸͙̠̼c̳̗͜o͏̼͙͔̮r̞̫̺̞̥̬ru̺̻̯͉̭̻̯p̰̥͓̣̫̙̤͢t̳͍̳̖ͅi̶͈̝͙̼̙̹o̡͔n̙̺̹̖̩͝ͅ”̨̗͖͚̩.̯͓  A few other places where dynamic character composition shows up in Unicode:  Vowel-pointing notation in Arabic and Hebrew. In these languages, words are normally spelled with some of their vowels left out. They then have diacritic notation to indicate the vowels (used in dictionaries, language-teaching materials, children’s books, and such). These diacritics are expressed with combining marks.  A Hebrew example, with niqqud:\tאֶת דַלְתִּי הֵזִיז הֵנִיעַ, קֶטֶב לִשְׁכַּתִּי יָשׁוֹד Normal writing (no niqqud):\tאת דלתי הזיז הניע, קטב לשכתי ישוד Devanagari, the script used to write Hindi, Sanskrit, and many other South Asian languages, expresses certain vowels as combining marks attached to consonant letters. For example, “ह” + “\u200bि” = “हि” (“h” + “i” = “hi”). Korean characters stand for syllables, but they are composed of letters called jamo that stand for the vowels and consonants in the syllable. While there are code points for precomposed Korean syllables, it’s also possible to dynamically compose them by concatenating their jamo. For example, “ᄒ” + “ᅡ” + “ᆫ” = “한” (“h” + “a” + “n” = “han”). Canonical Equivalence In Unicode, precomposed characters exist alongside the dynamic composition system. A consequence of this is that there are multiple ways to express “the same” string—different sequences of code points that result in the same user-perceived characters. For example, as we saw earlier, we can express the character “Á” either as the single code point U+00C1, or as the string of two code points U+0041 U+0301.  Another source of ambiguity is the ordering of multiple diacritics in a single character. Diacritic order matters visually when two diacritics apply to the same side of the base character, e.g. both above: “ǡ” (dot, then macron) is different from “ā̇” (macron, then dot). However, when diacritics apply to different sides of the character, e.g. one above and one below, then the order doesn’t affect rendering. Moreover, a character with multiple diacritics might have one of the diacritics precomposed and others expressed as combining marks.  For example, the Vietnamese letter “ệ” can be expressed in five different ways:  Fully precomposed: U+1EC7 “ệ” Partially precomposed: U+1EB9 “ẹ” + U+0302 “◌̂” Partially precomposed: U+00EA “ê” + U+0323 “◌̣” Fully decomposed: U+0065 “e” + U+0323 “◌̣” + U+0302 “◌̂” Fully decomposed: U+0065 “e” + U+0302 “◌̂” + U+0323 “◌̣” Unicode refers to set of strings like this as “canonically equivalent”. Canonically equivalent strings are supposed to be treated as identical for purposes of searching, sorting, rendering, text selection, and so on. This has implications for how you implement operations on text. For example, if an app has a “find in file” operation and the user searches for “ệ”, it should, by default, find occurrences of any of the five versions of “ệ” above!  Normalization Forms To address the problem of “how to handle canonically equivalent strings”, Unicode defines several normalization forms: ways of converting strings into a canonical form so that they can be compared code-point-by-code-point (or byte-by-byte).  The “NFD” normalization form fully decomposes every character down to its component base and combining marks, taking apart any precomposed code points in the string. It also sorts the combining marks in each character according to their rendered position, so e.g. diacritics that go below the character come before the ones that go above the character. (It doesn’t reorder diacritics in the same rendered position, since their order matters visually, as previously mentioned.)  The “NFC” form, conversely, puts things back together into precomposed code points as much as possible. If an unusual combination of diacritics is called for, there may not be any precomposed code point for it, in which case NFC still precomposes what it can and leaves any remaining combining marks in place (again ordered by rendered position, as in NFD).  There are also forms called NFKD and NFKC. The “K” here refers to compatibility decompositions, which cover characters that are “similar” in some sense but not visually identical. However, I’m not going to cover that here.  Grapheme Clusters As we’ve seen, Unicode contains various cases where a thing that a user thinks of as a single “character” might actually be made up of multiple code points under the hood. Unicode formalizes this using the notion of a grapheme cluster: a string of one or more code points that constitute a single “user-perceived character”.  UAX #29 defines the rules for what, precisely, qualifies as a grapheme cluster. It’s approximately “a base code point followed by any number of combining marks”, but the actual definition is a bit more complicated; it accounts for things like Korean jamo, and emoji ZWJ sequences.  The main thing grapheme clusters are used for is text editing: they’re often the most sensible unit for cursor placement and text selection boundaries. Using grapheme clusters for these purposes ensures that you can’t accidentally chop off some diacritics when you copy-and-paste text, that left/right arrow keys always move the cursor by one visible character, and so on.  Another place where grapheme clusters are useful is in enforcing a string length limit—say, on a database field. While the true, underlying limit might be something like the byte length of the string in UTF-8, you wouldn’t want to enforce that by just truncating bytes. At a minimum, you’d want to “round down” to the nearest code point boundary; but even better, round down to the nearest grapheme cluster boundary. Otherwise, you might be corrupting the last character by cutting off a diacritic, or interrupting a jamo sequence or ZWJ sequence.  And More… There’s much more that could be said about Unicode from a programmer’s perspective! I haven’t gotten into such fun topics as case mapping, collation, compatibility decompositions and confusables, Unicode-aware regexes, or bidirectional text. Nor have I said anything yet about implementation issues—how to efficiently store and look-up data about the sparsely-assigned code points, or how to optimize UTF-8 decoding, string comparison, or NFC normalization. Perhaps I’ll return to some of those things in future posts.  Unicode is a fascinating and complex system. It has a many-to-one mapping between bytes and code points, and on top of that a many-to-one (or, under some circumstances, many-to-many) mapping between code points and “characters”. It has oddball special cases in every corner. But no one ever claimed that representing all written languages was going to be easy, and it’s clear that we’re never going back to the bad old days of a patchwork of incompatible encodings.  Further reading:  The Unicode Standard UTF-8 Everywhere Manifesto Dark corners of Unicode by Eevee ICU (International Components for Unicode)—C/C++/Java libraries implementing many Unicode algorithms and related things Python 3 Unicode Howto Google Noto Fonts—set of fonts intended to cover all assigned code points"""
tokens = text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience

In [19]:
new_tokens, mapping = apply_bpe(tokens)

In [20]:
# Before, After
len(tokens), len(new_tokens)

(24597, 19438)

In [21]:
mapping

{(101, 32): 256,
 (105, 110): 257,
 (115, 32): 258,
 (116, 104): 259,
 (101, 114): 260,
 (99, 111): 261,
 (116, 32): 262,
 (226, 128): 263,
 (44, 32): 264,
 (97, 110): 265,
 (111, 114): 266,
 (100, 32): 267,
 (97, 114): 268,
 (101, 110): 269,
 (257, 103): 270,
 (261, 100): 271,
 (121, 32): 272,
 (46, 32): 273,
 (97, 108): 274,
 (259, 256): 275}

In [22]:
def get_compression_ratio(old, new):
    print("Old-Tokens length:", len(old))
    print("New-Tokens length:", len(new))
    print(f"Compression ratio: {len(old) / len(new):.2f}x")

In [23]:
get_compression_ratio(tokens, new_tokens)

Old-Tokens length: 24597
New-Tokens length: 19438
Compression ratio: 1.27x


**Interpretation**: After performing merges 20 times, we have achieved 1.27x compression.


> # 🔥
>
> *That was the **training** of the tokenizer!*

# 🔤🔄🔤 Encoder Decoder 
Let's start working on the encoder and decoder as that should work with the merges etc

## 1️⃣2️⃣3️⃣👉🏻🔤 Decoder

### 1️⃣ Step: Create a base vocab

In [24]:
# this will create the basic vocab for all **atomic** characters (available in the 256 range)
vocab = {idx:bytes([idx]) for idx in range(256)}

> ## 🟢
> 
> **NOTE**: These are the *default* characters that can be stored within single byte.
Luckily that covers all english characters and common special characters, and other "specially special characters" which used in combination can create any letter. So that's what we need.

Let me illustrate that to you.

In [25]:
# This is the simple byte character
vocab[76]

b'L'

In [26]:
# Converting that will give
vocab[76].decode("utf-8")

'L'

In [27]:
# This is the "special character" with its combination, we can create any letter
vocab[224]

b'\xe0'

In [28]:
# If we try to decode it (WILL GIVE ERROR)
try:
    vocab[224].decode("utf-8")
except Exception as e:
    print(">>>", e)

>>> 'utf-8' codec can't decode byte 0xe0 in position 0: unexpected end of data


👆🏻 Why? Because it **can't be used solely** we need to use it in combination.

In [29]:
# This is the "special character" with its combination, we can create any letter
(vocab[224] + vocab[170] + vocab[134]).decode("utf-8")

'આ'

In [30]:
# I got that by the following 👇🏻
list("આયુષ".encode("utf-8"))

[224, 170, 134, 224, 170, 175, 224, 171, 129, 224, 170, 183]

**Where:**

- "આ" = 224, 170, 134
- "ય" = 224, 170, 175
- 'ુ' = 224, 171, 129
- "ષ" = 224, 170, 183

> Because "utf-8" is just a representation of bytes, it can take 1 to 4 bytes per character! 😉

In [31]:
vocab

{0: b'\x00',
 1: b'\x01',
 2: b'\x02',
 3: b'\x03',
 4: b'\x04',
 5: b'\x05',
 6: b'\x06',
 7: b'\x07',
 8: b'\x08',
 9: b'\t',
 10: b'\n',
 11: b'\x0b',
 12: b'\x0c',
 13: b'\r',
 14: b'\x0e',
 15: b'\x0f',
 16: b'\x10',
 17: b'\x11',
 18: b'\x12',
 19: b'\x13',
 20: b'\x14',
 21: b'\x15',
 22: b'\x16',
 23: b'\x17',
 24: b'\x18',
 25: b'\x19',
 26: b'\x1a',
 27: b'\x1b',
 28: b'\x1c',
 29: b'\x1d',
 30: b'\x1e',
 31: b'\x1f',
 32: b' ',
 33: b'!',
 34: b'"',
 35: b'#',
 36: b'$',
 37: b'%',
 38: b'&',
 39: b"'",
 40: b'(',
 41: b')',
 42: b'*',
 43: b'+',
 44: b',',
 45: b'-',
 46: b'.',
 47: b'/',
 48: b'0',
 49: b'1',
 50: b'2',
 51: b'3',
 52: b'4',
 53: b'5',
 54: b'6',
 55: b'7',
 56: b'8',
 57: b'9',
 58: b':',
 59: b';',
 60: b'<',
 61: b'=',
 62: b'>',
 63: b'?',
 64: b'@',
 65: b'A',
 66: b'B',
 67: b'C',
 68: b'D',
 69: b'E',
 70: b'F',
 71: b'G',
 72: b'H',
 73: b'I',
 74: b'J',
 75: b'K',
 76: b'L',
 77: b'M',
 78: b'N',
 79: b'O',
 80: b'P',
 81: b'Q',
 82: b'R',
 83: b'

### 2️⃣ Step: Extend the vocab with new tokens

👉🏻 Now we will add the **memrged** or new created characters in the vocab. 

The vocab has the following structure:
```
{
    byte_index : byte_representation
}
```

Till now, we have the byte index from `0` to `255`. And, the following code will add new merged characters.

In [32]:
for (left, right), idx in mapping.items():
    vocab[idx] = vocab[left] + vocab[right]

In [33]:
# The old and new... all in the single vocab

print("{", *[(idx, b) for idx, b in vocab.items() if idx <= 4], "}", sep="\n")
print("\n...\n")
print("{", *[(idx, b) for idx, b in vocab.items() if idx >= 256 and idx <= 260], "}", sep="\n")

{
(0, b'\x00')
(1, b'\x01')
(2, b'\x02')
(3, b'\x03')
(4, b'\x04')
}

...

{
(256, b'e ')
(257, b'in')
(258, b's ')
(259, b'th')
(260, b'er')
}


**Note** that the `bytes([256])` won't work. Doing that will give an error like below:

```python
>>> byte([256])
ValueError: bytes must be in range(0, 256)
```

In [34]:
vocab

{0: b'\x00',
 1: b'\x01',
 2: b'\x02',
 3: b'\x03',
 4: b'\x04',
 5: b'\x05',
 6: b'\x06',
 7: b'\x07',
 8: b'\x08',
 9: b'\t',
 10: b'\n',
 11: b'\x0b',
 12: b'\x0c',
 13: b'\r',
 14: b'\x0e',
 15: b'\x0f',
 16: b'\x10',
 17: b'\x11',
 18: b'\x12',
 19: b'\x13',
 20: b'\x14',
 21: b'\x15',
 22: b'\x16',
 23: b'\x17',
 24: b'\x18',
 25: b'\x19',
 26: b'\x1a',
 27: b'\x1b',
 28: b'\x1c',
 29: b'\x1d',
 30: b'\x1e',
 31: b'\x1f',
 32: b' ',
 33: b'!',
 34: b'"',
 35: b'#',
 36: b'$',
 37: b'%',
 38: b'&',
 39: b"'",
 40: b'(',
 41: b')',
 42: b'*',
 43: b'+',
 44: b',',
 45: b'-',
 46: b'.',
 47: b'/',
 48: b'0',
 49: b'1',
 50: b'2',
 51: b'3',
 52: b'4',
 53: b'5',
 54: b'6',
 55: b'7',
 56: b'8',
 57: b'9',
 58: b':',
 59: b';',
 60: b'<',
 61: b'=',
 62: b'>',
 63: b'?',
 64: b'@',
 65: b'A',
 66: b'B',
 67: b'C',
 68: b'D',
 69: b'E',
 70: b'F',
 71: b'G',
 72: b'H',
 73: b'I',
 74: b'J',
 75: b'K',
 76: b'L',
 77: b'M',
 78: b'N',
 79: b'O',
 80: b'P',
 81: b'Q',
 82: b'R',
 83: b'

### 3️⃣ Step: Decode the incoming tokens

In [35]:
def decode(ids):
    # get byte representation from the vocab
    bytes_to_decode = [vocab[idx] for idx in ids]
    
    # merge them into a single byte stream
    bytes_to_decode = b"".join(bytes_to_decode)
    
    # decode them in the utf-8
    return bytes_to_decode.decode("utf-8")

In [36]:
# Example
print(decode([97]))
print(decode([97, 97, 121, 117, 115, 104]))

a
aayush


In [37]:
# but it fails here
try: 
    decode([128])
except Exception as e:
    print(">>>", e, "<<<")

>>> 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte <<<


As `bytes_stream.decode("utf-8")` expects the UTF-8 standard stream, but 128 doesn't specify that. For more information, the following is the response from **ChatGPT**:

---
1. **Encoding:** The `bytes([250])` operation creates a bytes object with a single byte value of 250, which is within the valid range of 0 to 255.

2. **Decoding:** When attempting to decode the bytes using `byte_encoded.decode("utf-8")`, Python encounters the byte `0xFA`, which doesn't form a valid UTF-8 character sequence due to its most significant bit being set. This leads to the `UnicodeDecodeError`.

To avoid this error, you can decode bytes using an encoding like `'latin-1'` or `'ISO-8859-1'`, which maps each byte value to the corresponding Unicode code point. For example:

```python
byte_encoded = bytes([250])
decoded_string = byte_encoded.decode("latin-1")
```

This way, each byte value is treated independently, preventing decoding errors for byte values above 127.

---

This is how it works 🤷🏻‍♂️

### 3️⃣ Step: Decode the incoming tokens (revised)

In [38]:
def decode(ids):
    # get byte representation from the vocab
    bytes_to_decode = [vocab[idx] for idx in ids]
    
    # merge them into a single byte stream
    bytes_to_decode = b"".join(bytes_to_decode)
    
    # decode them in the utf-8
    return bytes_to_decode.decode("utf-8", errors="replace") # replace with the unkown token

In [39]:
# Example
print(decode([97]))
print(decode([97, 97, 121, 117, 115, 104]))
print(decode([128]))

a
aayush
�


#### ❓ Why and when can it happen?
- The utf-8 represenation is the multi-byte representation.
- When the ids are wrongly placed, then the byte representation will be flawed.
- It happens when the model that we have trained, produces bad ids, which can't be decoded.
- That case, we will need this kind of replacement.


### Decoder All Together

In [40]:
# Training
vocab = {idx:bytes([idx]) for idx in range(256)}
for (left, right), idx in mapping.items():
    vocab[idx] = vocab[left] + vocab[right]
    
# Applying
def decode(ids):
    # get byte representation from the vocab
    bytes_to_decode = [vocab[idx] for idx in ids]
    
    # merge them into a single byte stream
    bytes_to_decode = b"".join(bytes_to_decode)
    
    # decode them in the utf-8
    return bytes_to_decode.decode("utf-8", errors="replace") # replace with the unkown token

## 🔤👉🏻1️⃣2️⃣3️⃣ Encoder 

In [41]:
def encode(text):
    # now we have the initial raw tokens (un-merged)
    tokens = list(text.encode("utf-8"))
    
    # we will need to merge them based on the understanding of previous merge (mappings)
    # the mapping **works top-down** so, first we will replace the first and then other
    while len(tokens) >= 2: # need to have at least 2 tokens
        _, _, stats = get_max_occuring_pair_and_stats(tokens)
    
        # now we have all the possible pairs and their counts in the `stats` variable
        # THIS LINE IS DEMYSTIFIED IN THE FOLLOWING CELLS
        pair = min(stats, key=lambda p: mapping.get(p, float("inf")))
        
        if pair not in mapping:
            break
        # Now we have a pair that can be replaced in the raw_tokens
        idx = mapping[pair]
        tokens = replace(tokens, pair, idx)
    
    return tokens

In [42]:
encode("This is me now!")

[84, 104, 105, 258, 105, 258, 109, 256, 110, 111, 119, 33]

## Demystifying the Encoder

<img src="./images/encoder-demystify">

In [43]:
# Ohh, it worked!
decode(encode("This is me now!"))

'This is me now!'

In [44]:
# Ohh, it worked!
decode(encode("Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽"))

'Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽'

# ❕❗The Dog Issue.

Andrej explains this [in this clip](https://youtube.com/clip/Ugkx_AxtO-E1iQOzoIPEiirwqi5FpUMfEQyu?si=rNtzvKAmEi8ek2MI) ✂️

**In short**:
- The problem arises when "the combination of unnecessary words become too frequent that they endup being a single token". 
- Doing this, can cause the model to learn different sementic meaning of them.

For an example:
- The word `hey!!!`, `hey!!`, `hey??!` are just the same *(they may provide different weight - but for the sake of this explanation, continue reading)*. 
- Having enough repetations of them can make them appear as a single token and the model has to learn its meaning.

So at the EOD, the model will have `hey`, `hey!!`, `hey!!!`, `hey??!`, `!!`, `!`, `?` as seperate tokens. While they **should not be**.

This is the problem and in the upcoming cells we wil try to solve this *(as GPT-2 researchers did while creating the GPT-2)*.

## 📏 Creating a regex to *seperate* these occurances
Let's take the regex from the notebook.

✂️ Andrej interprets the regex: [in this clip](https://youtube.com/clip/Ugkxlr-k4i1Sv2mcQrj82JiXqQw6zvi6JgHe?si=iKeegcRYG4CeHMpq).

In [45]:
import regex as re

In [46]:
gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

print(re.findall(gpt2pat, "Hello've world123 how's are you!!!?"))

['Hello', "'ve", ' world', '123', ' how', "'s", ' are', ' you', '!!!?']


See? This is cool. It will make the seperate words where it finds the things are not moving properly.

Let's take the `hey` example.

In [47]:
print(re.findall(gpt2pat, "Hey there! How is it goin'!!!?"))

['Hey', ' there', '!', ' How', ' is', ' it', ' goin', "'!!!?"]


✨ Looks neat!

### 🤔 Then?

> *It first splits the sentence into individual words (which are cleaned) and then they and all these elements of this list are processed **independently** by the tokenizer and all the results of the process, are concatenated.*
>
> — Andrej



### 👍🏻 The benefit?
You will only find the pairs, in the individual words so the **hey!!!** phenomenon will never occur!

# Playing around with GPT-2/4 tokenizers
✂️ [This clip](https://youtube.com/clip/UgkxjQUZrGHpqeW8cOWxyV0t7koZtI8zzNDN?si=6A0R2qVvm2GBhdxj) again, I am sorry for sharing the clips than explaining them, but these things are better well explained in the video by Andrej, than me repeating that same process here.

So please have a look at that clip and then we will start from the following cells.

In [57]:
# !wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/vocab.bpe
# !wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/encoder.json

In [58]:
import os, json

with open('./data/encoder.json', 'r') as f:
    encoder = json.load(f) # <--- ~equivalent to our "vocab"

with open('./data/vocab.bpe', 'r', encoding="utf-8") as f:
    bpe_data = f.read()
bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]
# ^---- ~equivalent to our "merges"

In [61]:
len(encoder.keys())

50257

🔥🔥 Highly recommend: [this is where I got a great realization](https://youtube.com/clip/UgkxY1FGyip3akDBANOrkZwuKDqkZYZYkmB1?si=9WfMVBcp0sZaZSLZ). 

Here, Andrej explains the story behind the number `50257`. It is very easy!!

# 🫡 Alright then!
Let's meet in the next book where we will try to train the tokenizer on the TS. Yeah, my swiftie mode will be activated *(and can get pretty wild 🐯)*.

# 😉 Coming from the notebook `2. Swiftie Exercise`?
Let's wrap up the lecture with the highlevel points.

### Sentencepiece
- Andrej explains how the sentencepiece works. 
- The library unlike tiktoken, allows to train on the new text.
- There are so many hyperparameters that we need to know about while using that library.
- Used by LLAMA and Mistral and other models.

### What should be the ideal vocab size?
- It should not be too high, because the rate token embedding vector will be undertrained.
- The models generally use 100k as an upper bound.
- High number of tokens will also make the longer tokens, encapsulating multiple tokens into the single one and hence, a lot of the information can be compressed and the model will have less time to learn the connections.
- Adding new custom tokens to the pretrained model is possible and is fairly simple.

### Gist Tokens
- Suppose working in a project which requires very long prompts.
- The prompt, most of the time is frozen.
- It uses a lot of context length.
- So, here **introduce the new tokens** and these represent the original prompt.
- Then the model training happens and now, model understand the indstructions and everything within these new tokens.

These are the gist tokens.

📃 [Learning to compress prompts with Gist tokens](https://arxiv.org/abs/2304.08467)

### Why tokenizers behave weird?
From [here ✂️](https://youtube.com/clip/UgkxqKy5g8yZ0xPDQZPT3BvIxW7ooTu0aNEE?si=opA3gmQuxzTAudYH) Andrej starts testing the issues with tokenizers. It is recommend to watch the stuff there. 

